In [1]:
import sklearn
import numpy as np
import pandas as pd

In [2]:
#import data
train = pd.read_csv('../../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean-train.csv')
train.drop(columns=["Unnamed: 0"], inplace=True)

In [3]:
#create features
align_feature = pd.read_csv('../../data/processed/features/alignment_feature.csv')
cosine_feature = pd.read_csv('../../data/processed/features/cosine_feature.csv')
bow_feature = pd.read_csv('../../data/processed/features/headline_BoWBigram.csv')
qmark_feature = pd.read_csv('../../data/processed/features/headline_Qmark.csv')
neg_alignment_feature = pd.read_csv('../../data/processed/features/neg_alignment_feature.csv')
root_dist_feature = pd.read_csv('../../data/processed/features/root_dist_min.csv')
svo_feature = pd.read_csv('../../data/processed/features/svo_Lexical.csv')

In [4]:
#target_map = {"for": 0, "observing": 1, "against": 2}

In [5]:
#train = train.replace({"articleHeadlineStance": target_map})

In [6]:
train["articleHeadlineStance"].value_counts()#,test["articleHeadlineStance"].value_counts()

for          992
observing    775
against      304
Name: articleHeadlineStance, dtype: int64

In [7]:
train = pd.merge(train, align_feature, on="articleId")
train = pd.merge(train, cosine_feature, on="articleId")
train = pd.merge(train, bow_feature, on="articleId")
train = pd.merge(train, qmark_feature, on="articleId")
train = pd.merge(train, neg_alignment_feature, on="articleId")
train = pd.merge(train, root_dist_feature, on="articleId")
# train = pd.merge(train, svo_feature, on="articleId")

In [8]:
#train["articleHeadlineStance"] = train["articleHeadlineStance"].astype("int")

In [9]:
# merge datasets and shuuffle with random seed
seed = 1234

train = sklearn.utils.shuffle(train, random_state=seed)
#test = sklearn.utils.shuffle(test, random_state=seed)


In [10]:
# create train and test

X_train = train.to_numpy()[:,5:]
Y_train = train["articleHeadlineStance"].values.reshape((-1,))

# from sklearn.model_selection import train_test_split
# X_Train, X_Test, Y_Train, Y_Test = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)


#X_test = test.to_numpy()[:,5:]
#Y_test = test["articleHeadlineStance"].values.reshape((-1,))

In [11]:
# Y_train = Y_Train
# X_train = X_Train.astype(float)
# X_val = X_Test.astype(float)
# Y_val = Y_Test

In [12]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Precision is how many times we idenity something as postive and it's actually positive.
Recall is in the number of positive cases, how many did we get right. 

In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
skf = StratifiedKFold(n_splits=10, )
skf.get_n_splits(X_train, Y_train)


train_folds = []
valid_folds = []
for i, (train_index, test_index) in enumerate(skf.split(X_train, Y_train)):
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    x_train_fold, x_valid_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_valid_fold = Y_train[train_index], Y_train[test_index]
    
    '''bc = np.bincount(y_train_fold)
    ii = np.nonzero(bc)[0]
    print("train set distribution")
    print(list(zip(ii,bc[ii])))
    bc = np.bincount(y_valid_fold)
    ii = np.nonzero(bc)[0]
    print("validation set distribution")
    print(list(zip(ii,bc[ii])))
    print("----------------------------")
    '''
    train_fold = np.concatenate((x_train_fold, y_train_fold.reshape(-1, 1)), axis=1)
    valid_fold = np.concatenate((x_valid_fold, y_valid_fold.reshape(-1, 1)), axis=1)
    train_folds.append(train_fold)
    valid_folds.append(valid_fold)

TRAIN: (1863,) TEST: (208,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)
TRAIN: (1864,) TEST: (207,)


In [16]:
max_iter = 10e2
C=0.8
penalty='l1'

# Normal Train and Test

from sklearn.linear_model import LogisticRegression


model = LogisticRegression(penalty=penalty, C=C, max_iter = max_iter, multi_class='auto', solver="liblinear")
running_acc=[]
running_f1 = []
for k,l in zip(train_folds,valid_folds):
    train_X = k[:,:-1]
    train_y = np.ravel(k[:,-1:])
    valid_X = l[:,:-1]
    valid_y = np.ravel(l[:,-1:])
    ypred = model.fit(train_X, train_y).predict(valid_X)
    f1 = f1_score(valid_y, ypred, average=None)
    print("Number of mislabeled points out of a total %d points : %d" % (valid_X.shape[0], (valid_y != ypred).sum()))
    print("Accuracy on test set: "+str(model.score(valid_X, valid_y)))
    print(f"F1 Score {f1}")
    running_acc.append(model.score(valid_X, valid_y))
    running_f1.append(f1)

Number of mislabeled points out of a total 208 points : 46
Accuracy on test set: 0.7788461538461539
F1 Score [0.69090909 0.83333333 0.73103448]
Number of mislabeled points out of a total 207 points : 52
Accuracy on test set: 0.748792270531401
F1 Score [0.68       0.80717489 0.68085106]
Number of mislabeled points out of a total 207 points : 46
Accuracy on test set: 0.7777777777777778
F1 Score [0.79310345 0.83257919 0.68148148]
Number of mislabeled points out of a total 207 points : 48
Accuracy on test set: 0.7681159420289855
F1 Score [0.83636364 0.79620853 0.7027027 ]
Number of mislabeled points out of a total 207 points : 43
Accuracy on test set: 0.7922705314009661
F1 Score [0.84615385 0.8125     0.73913043]
Number of mislabeled points out of a total 207 points : 45
Accuracy on test set: 0.782608695652174
F1 Score [0.73469388 0.81690141 0.75      ]
Number of mislabeled points out of a total 207 points : 47
Accuracy on test set: 0.7729468599033816
F1 Score [0.79245283 0.80751174 0.7162

In [17]:
print("mean acc " + str(np.array(running_acc).mean()))
print("std acc " + str(np.array(running_acc).std()))

mean acc 0.770638238573021
std acc 0.0132483005523329


In [18]:
against_f1 = [ i[0] for i in running_f1]
for_f1 = [ i[1] for i in running_f1]
observing_f1 = [ i[2] for i in running_f1]
print("mean agf1 " + str(np.array(against_f1).mean()))
print("std agf1 " + str(np.array(against_f1).std()))
print("mean forf1 " + str(np.array(for_f1).mean()))
print("std forf1 " + str(np.array(for_f1).std()))
print("mean obsf1 " + str(np.array(observing_f1).mean()))
print("std obsf1 " + str(np.array(observing_f1).std()))

mean agf1 0.7473391829157234
std agf1 0.06892650917804831
mean forf1 0.813088739813975
std forf1 0.011528967557117608
mean obsf1 0.7154321379386902
std obsf1 0.022322577140768067
